In [1]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

import numpy as np
from scipy.stats import rankdata

def inverse_percentile_rank(data, min_percentile=35, max_percentile=100):
    ranks = rankdata(data)
    min_rank = np.min(ranks)
    max_rank = np.max(ranks)
    scaled_ranks = ((max_percentile - min_percentile) * (1 - (ranks - min_rank) / (max_rank - min_rank))) + min_percentile
    return scaled_ranks

def percentile_rank(data, min_percentile = 40, max_percentile = 100):
    ranks = rankdata(data)
    min_rank = np.min(ranks)
    max_rank = np.max(ranks)
    scaled_ranks = ((ranks - min_rank) / (max_rank - min_rank)) * (max_percentile - min_percentile) + min_percentile
    return scaled_ranks

user="user"
password="password"
host='192.168.59.101'
port= '30432'
database="football-db"

db = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
conn = db.connect()

# Attack 

Attack Rating = (Goals Scored + Shots on Target + Possession) / 3


* Squad Shooting
    * Standard_Gls
    * Standard_SoT
* Squad Standard Stats
    * Poss

In [67]:
attack = pd.read_sql_query(sql="""SELECT t1.squad, standard_gls, standard_sot, poss FROM shooting_stats as t1 
                  INNER JOIN standard_stats as t2 ON t1.squad = t2.squad 
                                                    AND t1.value = t2.value where t1.value = 'squad' """, con=conn)



attack[[ "standard_gls", "standard_sot", "poss" ]] = attack[[ "standard_gls", "standard_sot", "poss" ]].apply(percentile_rank)

attack.loc[:, "attack"] =  attack[[ "standard_gls", "standard_sot", "poss" ]].mean(axis=1) / 3

attack

,squad,standard_gls,standard_sot,poss,attack
0,arsenal,96.842105,87.368421,90.526316,91.578947
1,aston villa,70.000000,71.578947,71.578947,71.052632
2,bournemouth,54.210526,46.315789,43.157895,47.894737
3,brentford,79.473684,74.736842,52.631579,68.947368
4,brighton,88.947368,100.000000,93.684211,94.210526
5,chelsea,54.210526,77.894737,87.368421,73.157895
6,crystal palace,58.947368,54.210526,58.947368,57.368421
7,everton,43.157895,68.421053,49.473684,53.684211
8,fulham,74.736842,58.947368,68.421053,67.368421
9,leeds united,65.263158,62.105263,62.105263,63.157895


# Defence 

Defense Rating = (Goals Conceded + Clean Sheets + Tackles) / 3

* defensive_action_stats
    * int
    * tackles_tkl
    * tackles_tklw
* goalkeeping_stats
    * performance_ga
    * performance_sota


In [5]:
defence = pd.read_sql_query(sql="""SELECT t1.squad, int, tackles_tkl, tackles_tklw, performance_ga, performance_sota FROM defensive_action_stats as t1 
                  INNER JOIN goalkeeping_stats as t2 ON t1.squad = t2.squad 
                                                    AND t1.value = t2.value where t1.value = 'squad' """, con=conn)

defence.loc[:, "tackles_win_pct"] = (defence.loc[:, "tackles_tklw"].astype('float16') / defence.loc[:, "tackles_tkl"].astype('float16')) * 100

defence[[ "tackles_win_pct", "int" ]] = defence[[ "tackles_win_pct", "int" ]].apply(percentile_rank)
defence[[ "performance_ga", "performance_sota" ]] = defence[[ "performance_ga", "performance_sota" ]].apply(inverse_percentile_rank)

defence.loc[:, "defence"] =  defence[[ "tackles_win_pct", "int", "performance_ga", "performance_sota" ]].mean(axis=1)

defence

,squad,int,tackles_tkl,tackles_tklw,performance_ga,performance_sota,tackles_win_pct,defence
0,arsenal,43.157895,568,343,92.972973,89.736842,90.526316,79.098506
1,aston villa,52.631579,633,338,85.945946,79.473684,40.000000,64.512802
2,bournemouth,81.052632,618,371,42.027027,35.000000,84.210526,60.572546
3,brentford,70.000000,587,337,85.945946,38.421053,62.105263,64.118065
4,brighton,49.473684,616,340,68.378378,93.157895,44.736842,63.936700
5,chelsea,70.000000,740,445,78.918919,72.631579,87.368421,77.229730
6,crystal palace,84.210526,690,421,73.648649,69.210526,96.842105,80.977952
7,everton,90.526316,708,424,59.594595,46.973684,81.052632,69.536807
8,fulham,63.684211,623,349,68.378378,41.842105,52.631579,56.634068
9,leeds united,93.684211,840,480,35.000000,46.973684,58.947368,58.651316


# Midfield

Midfield Rating = (Pass Completion + Key Passes + Interceptions) / 3

* passing_types_stats
    * pass_types_crs
    * outcomes_off
    * outcomes_blocks
* passing_stats
    * ast
    * kp

In [89]:
midfield = pd.read_sql_query(sql="""SELECT t1.squad, passtypes_crs, outcomes_off, outcomes_blocks, ast, kp FROM passing_types_stats as t1 
                  INNER JOIN passing_stats as t2 ON t1.squad = t2.squad 
                                                    AND t1.value = t2.value where t1.value = 'squad' """, con=conn)

midfield[[ "passtypes_crs", "ast", "kp" ]] = midfield[[ "passtypes_crs", "ast", "kp" ]].apply(percentile_rank)
midfield[[ "outcomes_off", "outcomes_blocks" ]] = midfield[[ "outcomes_off", "outcomes_blocks" ]].apply(inverse_percentile_rank)

midfield.loc[:, "midfield"] =  midfield[[ "outcomes_off", "outcomes_blocks", "passtypes_crs", "ast", "kp" ]].mean(axis=1)
midfield

,squad,passtypes_crs,outcomes_off,outcomes_blocks,ast,kp,midfield
0,arsenal,71.578947,93.157895,62.368421,96.842105,87.368421,82.263158
1,aston villa,52.631579,84.605263,89.736842,71.578947,65.263158,72.763158
2,bournemouth,43.157895,100.000000,86.315789,47.894737,43.157895,64.105263
3,brentford,66.842105,48.684211,100.000000,76.315789,46.315789,67.631579
4,brighton,77.894737,55.526316,57.236842,88.947368,100.000000,75.921053
5,chelsea,74.736842,45.263158,69.210526,58.947368,77.894737,65.210526
6,crystal palace,55.789474,84.605263,81.184211,62.105263,62.105263,69.157895
7,everton,81.052632,65.789474,96.578947,47.894737,58.947368,70.052632
8,fulham,95.263158,77.763158,46.973684,68.421053,55.789474,68.842105
9,leeds united,62.105263,77.763158,40.131579,65.263158,71.578947,63.368421


In [92]:
data = pd.merge(attack[["squad", "attack"]], midfield[["squad", "midfield"]], on='squad').merge(defence[["squad", "defence"]], on='squad' )


In [94]:
data.loc[:, "overall"] = data[[ "attack", "midfield", "defence" ]].mean(axis=1)

In [3]:
pd.read_sql_query(sql="""SELECT * from ratings""", con=conn)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "ratings" does not exist
LINE 1: SELECT * from ratings
                      ^

[SQL: SELECT * from ratings]
(Background on this error at: https://sqlalche.me/e/14/f405)

SyntaxError: invalid syntax (<ipython-input-102-b3f8f88106b2>, line 1)

In [4]:
??db.connect

Signature: db.connect(close_with_result=False)
Source:   
    def connect(self, close_with_result=False):
        """Return a new :class:`_engine.Connection` object.

        The :class:`_engine.Connection` object is a facade that uses a DBAPI
        connection internally in order to communicate with the database.  This
        connection is procured from the connection-holding :class:`_pool.Pool`
        referenced by this :class:`_engine.Engine`. When the
        :meth:`_engine.Connection.close` method of the
        :class:`_engine.Connection` object
        is called, the underlying DBAPI connection is then returned to the
        connection pool, where it may be used again in a subsequent call to
        :meth:`_engine.Engine.connect`.

        """

        return self._connection_cls(self, close_with_result=close_with_result)
File:      c:\users\saiha\anaconda3\lib\site-packages\sqlalchemy\engine\base.py
Type:      method


In [5]:
type(db)

sqlalchemy.engine.base.Engine